In [ ]:
import pandas as pd
import numpy as np
import os
import gzip
import time
import zipfile
from scipy.stats import entropy, ks_2samp, f_oneway
from scipy.special import rel_entr
from statsmodels.nonparametric.smoothers_lowess import lowess
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture
from scipy.spatial.distance import jensenshannon

from scipy.interpolate import interp1d


# Function to calculate KL divergence
def kl_divergence(p, q):
    return np.sum(rel_entr(p, q))

# Function to calculate geometric Jensen-Shannon divergence
def geometric_jsd(p, q):
    jsd = jensenshannon(p, q)
    return np.sqrt(jsd)

# Function to perform Kolmogorov-Smirnov test
def ks_test(p, q):
    ks_result = ks_2samp(p, q)
    return ks_result.statistic, ks_result.pvalue

# Smoothing functions
def moving_average(data, window_size=3):
    return np.convolve(data, np.ones(window_size)/window_size, mode='valid')

def gaussian_smoothing(data, sigma=10):  # Adjusted sigma for smaller range
    return np.exp(-np.square(data - np.mean(data)) / (2 * sigma**2)) / (sigma * np.sqrt(2 * np.pi))

def laplace_smoothing(data, alpha=1e-10):  # Much smaller alpha for more realistic smoothing
    return (data + alpha) / (np.sum(data) + alpha * len(data))

def good_turing_smoothing(data):
    unique, counts = np.unique(data, return_counts=True)
    freq_of_freqs = np.bincount(counts)
    smoothed = np.array([((c+1) * (freq_of_freqs[c+1]/freq_of_freqs[c]) if c+1 < len(freq_of_freqs) else c) for c in counts])
    return smoothed / np.sum(smoothed)

def lowess_smoothing(data, frac=0.1, downsample_factor=10):
    n = len(data)
    
    # Downsample the data
    downsampled_indices = np.arange(0, n, downsample_factor)
    downsampled_data = data[downsampled_indices]
    
    smoothed_downsampled_data = lowess(downsampled_data, downsampled_indices, frac=frac)[:, 1]
    
    interpolation_function = interp1d(downsampled_indices, smoothed_downsampled_data, kind='linear', fill_value="extrapolate")
    smoothed_data = interpolation_function(np.arange(n))
    
    return smoothed_data

def hmm_smoothing(data, n_components=2):
    model = GaussianMixture(n_components=n_components, max_iter=100)  # Reduced max_iter for faster computation
    data_reshaped = data.reshape(-1, 1)
    model.fit(data_reshaped)
    smoothed = model.predict_proba(data_reshaped)[:, 1]
    return smoothed

# Function to apply smoothing
def apply_smoothing(data, method):
    if method == 'moving_average':
        return moving_average(data)
    elif method == 'gaussian':
        return gaussian_smoothing(data)
    elif method == 'laplace':
        return laplace_smoothing(data)
    elif method == 'good_turing':
        return good_turing_smoothing(data)
    elif method == 'lowess':
        return lowess_smoothing(data)
    elif method == 'hmm':
        return hmm_smoothing(data)
    elif method == 'none':
        return data
    else:
        raise ValueError(f"Unknown smoothing method: {method}")

# Function to calculate divergence metrics
def divergence_calculations(data, smoothing='laplace'):
    # Filter out zero values in methylated and unmethylated counts
    data = data[(data['methylated'] > 0) & (data['unmethylated'] > 0)]

    # Calculate the statistics for the entire dataset
    p = data[['methylated', 'unmethylated']].values.flatten().astype(np.float64)
    q = data[['unmethylated', 'methylated']].values.flatten().astype(np.float64)

    # Apply smoothing if not 'none'
    if smoothing != 'none':
        p = apply_smoothing(p, smoothing)
        q = apply_smoothing(q, smoothing)

    # Normalize p and q
    p /= p.sum()
    q /= q.sum()

    kl = kl_divergence(p, q)
    js = jensenshannon(p, q)
    gjs = geometric_jsd(p, q)
    ks_stat, ks_pvalue = ks_test(p, q)

    dataset_result = {
        'entropy': entropy(p),
        'relative_entropy': kl,  # KL divergence is relative entropy
        'jsd': js,
        'geometric_jsd': gjs,
        'kolmogorov_smirnov_stat': ks_stat,
        'kolmogorov_smirnov_pvalue': ks_pvalue
    }
    return dataset_result

# Function to process input files
def process_file(file_path):
    with gzip.open(file_path, 'rt') as f:
        df = pd.read_csv(f, sep='\t', header=None, names=['chr', 'start', 'end', 'methylated', 'unmethylated', 'percentage'])
    return df

# Function to analyze samples
def analyze_samples(base_dir, smoothing_methods, save_interval=100):
    results = {}
    times = {method: [] for method in smoothing_methods}

    file_counter = 0
    for root, dirs, files in os.walk(base_dir):
        for file in files:
            if file.endswith('.cov.gz'):
                file_path = os.path.join(root, file)
                #print(f"Processing file: {file_path}")
                data = process_file(file_path)
                
                for method in smoothing_methods:
                    #print(f"Testing method: {method}")
                    start_time = time.time()
                    result = divergence_calculations(data, smoothing=method)
                    end_time = time.time()
                    elapsed_time = end_time - start_time
                    times[method].append(elapsed_time)
                    results[(file, method)] = result
                
                file_counter += 1

                # Save interim results periodically
                if file_counter % save_interval == 0:
                    save_results(results, times, 'interim_results.zip')

    # Save the final results
    save_results(results, times, 'final_results.zip')
    return results, times

# Function to save results to a zip file
def save_results(results, times, zip_file):
    results_df = pd.DataFrame(results).T
    results_df.to_csv('results.csv', index=True)
    
    times_df = pd.DataFrame(times)
    times_df.to_csv('times.csv', index=True)

    with zipfile.ZipFile(zip_file, 'w') as zipf:
        zipf.write('results.csv')
        zipf.write('times.csv')

    os.remove('results.csv')
    os.remove('times.csv')

# Function to plot results
def plot_results(results, times):
    methods = list(set(key[1] for key in results.keys()))
    files = list(set(key[0] for key in results.keys()))

    entropies = {method: [] for method in methods}
    kl_divergences = {method: [] for method in methods}
    jsds = {method: [] for method in methods}
    geometric_jsds = {method: [] for method in methods}
    ks_statistics = {method: [] for method in methods}
    ks_pvalues = {method: [] for method in methods}

    for file in files:
        for method in methods:
            result = results.get((file, method))
            if result:
                entropies[method].append(result['entropy'])
                kl_divergences[method].append(result['relative_entropy'])
                jsds[method].append(result['jsd'])
                geometric_jsds[method].append(result['geometric_jsd'])
                ks_statistics[method].append(result['kolmogorov_smirnov_stat'])
                ks_pvalues[method].append(result['kolmogorov_smirnov_pvalue'])

    x = np.arange(len(files))

    fig, axs = plt.subplots(3, 2, figsize=(20, 20))  # Increased figure size

    for method in methods:
        axs[0, 0].plot(x, entropies[method], label=method)
        axs[0, 1].plot(x, kl_divergences[method], label=method)
        axs[1, 0].plot(x, jsds[method], label=method)
        axs[1, 1].plot(x, geometric_jsds[method], label=method)
        axs[2, 0].plot(x, ks_statistics[method], label=method)
        axs[2, 1].plot(x, ks_pvalues[method], label=method)

    axs[0, 0].set_title('Entropy')
    axs[0, 1].set_title('KL Divergence')
    axs[1, 0].set_title('Jensen-Shannon Divergence')
    axs[1, 1].set_title('Geometric JSD')
    axs[2, 0].set_title('Kolmogorov-Smirnov Statistic')
    axs[2, 1].set_title('Kolmogorov-Smirnov P-value')

    for ax in axs.flat:
        ax.set_xticks(x[::max(1, len(x))])  # Show a subset of labels to avoid clutter
        ax.set_xticklabels(files[::max(1, len(x))], rotation=90)  # Rotate labels for better readability
        ax.legend()

    plt.tight_layout()
    plt.show()

    # Plotting the times for each method
    fig, ax = plt.subplots(figsize=(15, 7))
    avg_times = {method: np.mean(times[method]) for method in times}
    ax.bar(avg_times.keys(), avg_times.values())
    ax.set_title('Average Processing Time by Smoothing Method')
    ax.set_ylabel('Time (seconds)')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

    # ANOVA test to compare times between different smoothing methods
    time_values = [times[method] for method in times]
    anova_result = f_oneway(*time_values)
    print("ANOVA test result:", anova_result)

# Sample directory path
base_dir = '/home/eharpu/methylation_analysis/samples_testing'
smoothing_methods = ['none', 'gaussian', 'good_turing', 'hmm', 'lowess']
results, times = analyze_samples(base_dir, smoothing_methods)
plot_results(results, times)
